In [1]:
import numpy as np 
import pandas as pd 
import plotly.express as px

import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
train_data = pd.read_csv('/kaggle/input/playground-series-s4e2/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e2/test.csv')

pd.concat([test_data, train_data]).info(memory_usage=False)

<class 'pandas.core.frame.DataFrame'>
Index: 34598 entries, 0 to 20757
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              34598 non-null  int64  
 1   Gender                          34598 non-null  object 
 2   Age                             34598 non-null  float64
 3   Height                          34598 non-null  float64
 4   Weight                          34598 non-null  float64
 5   family_history_with_overweight  34598 non-null  object 
 6   FAVC                            34598 non-null  object 
 7   FCVC                            34598 non-null  float64
 8   NCP                             34598 non-null  float64
 9   CAEC                            34598 non-null  object 
 10  SMOKE                           34598 non-null  object 
 11  CH2O                            34598 non-null  float64
 12  SCC                             34598

In [3]:
train_data.describe().T

,count,mean,std,min,25%,50%,75%,max
id,20758.0,10378.500000,5992.462780,0.00,5189.250000,10378.500000,15567.750000,20757.000000
Age,20758.0,23.841804,5.688072,14.00,20.000000,22.815416,26.000000,61.000000
Height,20758.0,1.700245,0.087312,1.45,1.631856,1.700000,1.762887,1.975663
Weight,20758.0,87.887768,26.379443,39.00,66.000000,84.064875,111.600553,165.057269
FCVC,20758.0,2.445908,0.533218,1.00,2.000000,2.393837,3.000000,3.000000
NCP,20758.0,2.761332,0.705375,1.00,3.000000,3.000000,3.000000,4.000000
CH2O,20758.0,2.029418,0.608467,1.00,1.792022,2.000000,2.549617,3.000000
FAF,20758.0,0.981747,0.838302,0.00,0.008013,1.000000,1.587406,3.000000
TUE,20758.0,0.616756,0.602113,0.00,0.000000,0.573887,1.000000,2.000000


In [4]:
train_data.describe(include='O').T

,count,unique,top,freq
Gender,20758,2,Female,10422
family_history_with_overweight,20758,2,yes,17014
FAVC,20758,2,yes,18982
CAEC,20758,4,Sometimes,17529
SMOKE,20758,2,no,20513
SCC,20758,2,no,20071
CALC,20758,3,Sometimes,15066
MTRANS,20758,5,Public_Transportation,16687
NObeyesdad,20758,7,Obesity_Type_III,4046


In [5]:
fig = px.histogram(train_data.NObeyesdad, orientation='h', height=250)
fig.show()

In [6]:
corr = train_data.drop(columns=['id']).corr(numeric_only=True)
fig = px.imshow(corr, text_auto=True, aspect='auto')
fig.update_xaxes(side="top")
fig.show()

In [7]:
def get_fitted_catboost(params): 
    X_train, X_test, y_train, y_test = train_test_split(train_data.drop(columns=['NObeyesdad']), 
                                                        train_data.NObeyesdad, 
                                                        test_size=0.3, 
                                                        random_state=42) 
    cat_features = train_data.select_dtypes('O').columns.values[:-1]

    clf = CatBoostClassifier(**params, 
                             loss_function='MultiClass',
                             auto_class_weights='Balanced',
                             iterations=100, 
                             random_state=42)
    clf.fit(X_train,y_train, 
            eval_set=(X_test,y_test),
            early_stopping_rounds=100,
            cat_features=cat_features, 
            verbose=False)

    return clf, X_test, y_test

In [8]:
def objective(trial: optuna.Trial) -> float:
    params = {
        'learning_rate': trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
    }
    clf, X_test, y_test = get_fitted_catboost(params)
    
    predictions = clf.predict(X_test)
    return accuracy_score(y_test, predictions)

In [9]:
study = optuna.create_study(study_name="obesity-accuracy", direction="maximize")
study.optimize(objective, n_trials=30)

best_params = study.best_trial.params
print('Trials completed:', len(study.trials))
print("Best params: ", best_params)

Trials completed: 30
Best params:  {'learning_rate': 0.0976684330139157, 'colsample_bylevel': 0.08862455559521171, 'depth': 11, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli'}


In [10]:
clf = get_fitted_catboost(best_params)[0]
predictions = clf.predict(test_data)

submission = pd.read_csv('/kaggle/input/playground-series-s4e2/sample_submission.csv', index_col='id')
submission['NObeyesdad'] = predictions[:,0]
submission.to_csv('submission.csv')
submission.head()

,NObeyesdad
id,
20758,Obesity_Type_II
20759,Overweight_Level_I
20760,Obesity_Type_III
20761,Obesity_Type_I
20762,Obesity_Type_III
